In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "seaborn"

In [5]:
df = pd.read_csv('breastcancer.csv', header=None)